# Predict BertForTokenClassification - HAILab NER

In [6]:
import torch
from transformers import BertTokenizer,BertForTokenClassification
import numpy as np
import json

In [7]:
def predictBERTNER(sentencas,MODEL_DIR):
        
    model = BertForTokenClassification.from_pretrained(MODEL_DIR)
    tokenizer = BertTokenizer.from_pretrained(MODEL_DIR, do_lower_case=True) # lower or not, this is important

    with open(MODEL_DIR + '/idx2tag.json', 'r') as filehandle:
        idx2tag = json.load(filehandle) 
        
    predictedModel=[]
    
    for test_sentence in sentencas:
        tokenized_sentence = tokenizer.encode(test_sentence)
        input_ids = torch.tensor([tokenized_sentence])#.cuda() 
        
        with torch.no_grad():
            output = model(input_ids)
        label_indices = np.argmax(output[0].to('cpu').numpy(), axis=2)
        print(label_indices)
        
        # join bpe split tokens
        tokens = tokenizer.convert_ids_to_tokens(input_ids.to('cpu').numpy()[0])
        new_tokens, new_labels = [], []
        for token, label_idx in zip(tokens, label_indices[0]):
            if token.startswith("##"):
                new_tokens[-1] = new_tokens[-1] + token[2:]
            else:
                new_labels.append(label_idx)
                new_tokens.append(token)
            
        FinalLabelSentence = []
        for token, label in zip(new_tokens, new_labels):
            print(label)
            label = idx2tag[str(label)]
            if label == "O" or label == "X":
                FinalLabelSentence.append("O")
            else:
                FinalLabelSentence.append(label)
            
            
            print(token,label)
            print(FinalLabelSentence)
            print("+"*20)    
        
                
        predictedModel.append(FinalLabelSentence[1:-1]) # delete [SEP] and [CLS]
        
            
    return predictedModel

In [8]:
import nltk    
from nltk import tokenize    

# THE MODEL ACCEPTS ONLY LOWER
test_sentence1 = """Um paciente do sexo masculino, identificado como Zé, com 23, foi admitido no hospital em 22 de abril de 2024 após sofrer um traumatismo craniano grave em um acidente de trânsito. No local do incidente, ele foi encontrado inconsciente e exibia sinais de lesão grave na cabeça.
No exame físico inicial, o paciente estava em estado de coma profundo, com uma pontuação na Escala de Coma de Glasgow de 3, não respondendo a estímulos verbais, motores ou pupilares. Suas pupilas não reagiam à luz. Apesar disso, mantinha uma respiração espontânea, porém com possíveis alterações nos padrões respiratórios, e suas condições vitais como pressão arterial, frequência cardíaca e temperatura estavam dentro dos limites normais.""".lower()
test_sentence2 = "Paciente apresenta coriza intensa, necessário desentupir as vias nasais".lower()

test_sentence_tokenized = [tokenize.word_tokenize(test_sentence1, language='portuguese'),tokenize.word_tokenize(test_sentence2, language='portuguese')] 
print(test_sentence_tokenized)

[['um', 'paciente', 'do', 'sexo', 'masculino', ',', 'identificado', 'como', 'zé', ',', 'com', '23', ',', 'foi', 'admitido', 'no', 'hospital', 'em', '22', 'de', 'abril', 'de', '2024', 'após', 'sofrer', 'um', 'traumatismo', 'craniano', 'grave', 'em', 'um', 'acidente', 'de', 'trânsito', '.', 'no', 'local', 'do', 'incidente', ',', 'ele', 'foi', 'encontrado', 'inconsciente', 'e', 'exibia', 'sinais', 'de', 'lesão', 'grave', 'na', 'cabeça', '.', 'no', 'exame', 'físico', 'inicial', ',', 'o', 'paciente', 'estava', 'em', 'estado', 'de', 'coma', 'profundo', ',', 'com', 'uma', 'pontuação', 'na', 'escala', 'de', 'coma', 'de', 'glasgow', 'de', '3', ',', 'não', 'respondendo', 'a', 'estímulos', 'verbais', ',', 'motores', 'ou', 'pupilares', '.', 'suas', 'pupilas', 'não', 'reagiam', 'à', 'luz', '.', 'apesar', 'disso', ',', 'mantinha', 'uma', 'respiração', 'espontânea', ',', 'porém', 'com', 'possíveis', 'alterações', 'nos', 'padrões', 'respiratórios', ',', 'e', 'suas', 'condições', 'vitais', 'como', 'pre

In [9]:
MODEL_DIR = r"C:/Users/User/Desktop/Aulas PUC/7º semestre/Processamento de linguagem natural (NLP)/biobert-all-clinpt"
tags = predictBERTNER(test_sentence_tokenized,MODEL_DIR)
tags

[[14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14  3 24 24 14 14
   3 14 14 18  7  7 14 14 18  7  7 14 14 14 14 18 14 14 14 14 18 14 18  7
   7  7  7 14  8 14 14 12 20 14 14 14 14 14 14 18  7  7  7 14 14 14 18 14
  12 20 20 14 20 14  6 14  5 17 14 17 14 14 17 14 17 14 14 18  5 17 14 17
  14 14 14 14 14 14 18  7 14 14 14 17  7 15 15 14 14 14 14 18 14 14 17 14
  14 17  7 14 17 14 17 15 15 14 14 14]]
14
[CLS] O
['O']
++++++++++++++++++++
14
um O
['O', 'O']
++++++++++++++++++++
14
paciente O
['O', 'O', 'O']
++++++++++++++++++++
14
do O
['O', 'O', 'O', 'O']
++++++++++++++++++++
14
sexo O
['O', 'O', 'O', 'O', 'O']
++++++++++++++++++++
14
masculino O
['O', 'O', 'O', 'O', 'O', 'O']
++++++++++++++++++++
14
, O
['O', 'O', 'O', 'O', 'O', 'O', 'O']
++++++++++++++++++++
14
[UNK] O
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
++++++++++++++++++++
14
como O
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
++++++++++++++++++++
14
[UNK] O
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
+++++++++++

[['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-DT',
  'I-DT',
  'I-DT',
  'O',
  'O',
  'B-DT',
  'O',
  'O',
  'B-C',
  'I-C',
  'I-C',
  'O',
  'O',
  'B-C',
  'I-C',
  'I-C',
  'O',
  'O',
  'O',
  'O',
  'B-C',
  'O',
  'O',
  'O',
  'O',
  'B-C',
  'O',
  'B-C',
  'I-C',
  'I-C',
  'I-C',
  'I-C',
  'O',
  'B-AS',
  'O',
  'O',
  'B-T',
  'I-T',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-C',
  'I-C',
  'I-C',
  'I-C',
  'O',
  'O',
  'O',
  'B-C',
  'O',
  'B-T',
  'I-T',
  'I-T',
  'O',
  'I-T',
  'O',
  'B-V',
  'O',
  'B-N',
  'B-R',
  'O',
  'B-R',
  'O',
  'O',
  'B-R',
  'O',
  'B-R',
  'O',
  'O',
  'B-C',
  'B-N',
  'B-R',
  'O',
  'B-R',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-C',
  'I-C',
  'O',
  'O',
  'O',
  'B-R',
  'I-C',
  'I-R',
  'I-R',
  'O',
  'O',
  'O',
  'O',
  'B-C',
  'O',
  'O',
  'B-R',
  'O',
  'O',
  'B-R',
  'I-C',
  'O',
  'B-R',
  'O',
  'B-R',
  'I-R',
  

In [10]:
tags[1]

['O', 'O', 'B-C', 'I-C', 'O', 'O', 'B-C', 'O', 'I-C', 'O']

In [11]:
for s,sa in zip(test_sentence_tokenized,tags):
    for t, a in zip(s,sa):
        print(f"{t} - {a}")

um - O
paciente - O
do - O
sexo - O
masculino - O
, - O
identificado - O
como - O
zé - O
, - O
com - O
23 - O
, - O
foi - O
admitido - O
no - O
hospital - O
em - O
22 - B-DT
de - I-DT
abril - I-DT
de - O
2024 - O
após - B-DT
sofrer - O
um - O
traumatismo - B-C
craniano - I-C
grave - I-C
em - O
um - O
acidente - B-C
de - I-C
trânsito - I-C
. - O
no - O
local - O
do - O
incidente - B-C
, - O
ele - O
foi - O
encontrado - O
inconsciente - B-C
e - O
exibia - B-C
sinais - I-C
de - I-C
lesão - I-C
grave - I-C
na - O
cabeça - B-AS
. - O
no - O
exame - B-T
físico - I-T
inicial - O
, - O
o - O
paciente - O
estava - O
em - O
estado - B-C
de - I-C
coma - I-C
profundo - I-C
, - O
com - O
uma - O
pontuação - B-C
na - O
escala - B-T
de - I-T
coma - I-T
de - O
glasgow - I-T
de - O
3 - B-V
, - O
não - B-N
respondendo - B-R
a - O
estímulos - B-R
verbais - O
, - O
motores - B-R
ou - O
pupilares - B-R
. - O
suas - O
pupilas - B-C
não - B-N
reagiam - B-R
à - O
luz - B-R
. - O
apesar - O
disso - O
, - O
man

In [12]:
#Reference: 
#CH: Characterization; 
#T: Test; 
#EV: Evolution; 
#G: Genetics; 
#AS: Anatomical Site; 
#N: Negation; 
#OBS: Additional Observations; 
#C: Condition; 
#R: Results; 
#DT: DateTime; 
#THER: Therapeutics; 
#V: Value; 
#RA: Route of Administration; 
#O: Out